In [1]:
import tomopy
import math
import dxchange
import os
import matplotlib.pyplot as plt
import numpy as np
import bm3d_streak_removal as bm3d_rmv
from imars3dv2.filters import tilt
# import logging
# logging.basicConfig(level=logging.INFO)

# %matplotlib notebook

olefile module not found


In [ ]:
def get_ind_list(name_list:list):
    ind = []
    ang_deg = []
    ang_rad = []
    ind_dict_random = {}
    ind_dict_sorted = {}
    for e_name in name_list:
        _split = e_name.split('_')
        _index_tiff = _split[-1]
        _index = _index_tiff.split('.')[0]
        _ang = _split[-3] + '.' + _split[-2]
        index = int(_index)
        angle = float(_ang)
        ind.append(index)
        ang_deg.append(angle)
        ang_rad.append(math.radians(angle))
        ind_dict_random[index] = e_name
    ind = sorted(ind)
    for n, e_ind in enumerate(ind):
        ind_dict_sorted[n] = ind_dict_random[e_ind]
    
    return list(ind_dict_sorted.values()), (sorted(ang_deg)), np.array(sorted(ang_rad)), ind

def get_list(name_list:list):
    ind = []
    ind_dict_random = {}
    ind_dict_sorted = {}
    for e_name in name_list:
        _split = e_name.split('_')
        _index_tiff = _split[-1]
        _index = _index_tiff.split('.')[0]
        index = int(_index)
        ind.append(index)
        ind_dict_random[index] = e_name
    ind = sorted(ind)
    for n, e_ind in enumerate(ind):
        ind_dict_sorted[n] = ind_dict_random[e_ind]
    
    return list(ind_dict_sorted.values()), ind

def _init_arr_from_stack(fname, number_of_files, slc=None):
    """
    Initialize numpy array from files in a folder.
    """
    _arr = dxchange.read_tiff(fname, slc)
    size = (number_of_files, _arr.shape[0], _arr.shape[1])
    return np.empty(size, dtype=_arr.dtype)

def read_tiff_stack(fdir, fname:list):
    arr = _init_arr_from_stack(os.path.join(fdir, fname[0]), len(fname))
    for m, name in enumerate(fname):
        arr[m] = dxchange.read_tiff(os.path.join(fdir, name))
    return arr

def find_proj180_ind(ang_list:list):
    dif = [abs(x-180) for x in ang_list]
    difmin = min(dif)
    ind180 = dif.index(difmin)
    return (ind180, ang_list[ind180])

def shrink_window(corners, size):
    corners[0][0] = corners[0][0] + size
    corners[0][1] = corners[0][1] + size
    corners[1][0] = corners[1][0] + size
    corners[1][1] = corners[1][1] - size
    corners[2][0] = corners[2][0] - size
    corners[2][1] = corners[2][1] - size
    corners[3][0] = corners[3][0] - size
    corners[3][1] = corners[3][1] + size
    return corners

def set_roi(corners, xmin, ymin, xmax, ymax):
    corners[0][0] = xmin
    corners[0][1] = ymin
    corners[1][0] = xmin
    corners[1][1] = ymax
    corners[2][0] = xmax
    corners[2][1] = ymax
    corners[3][0] = xmax
    corners[3][1] = ymin
    return corners

In [ ]:
###################### need to change start
ipts="23788"
###################### need to change end

In [ ]:
# ct_scan_loc = "/HFIR/CG1D/IPTS-"+ipts+"/raw/ct_scans"
ct_scan_loc = "/Users/j35/IPTS/HFIR/CG1D/IPTS-27158/raw/ct_scans/"
os.listdir(ct_scan_loc)

In [ ]:
###################### need to change start
ct_scan_dir = os.path.join(ct_scan_loc, "ct_scans_files_combined_by_15_median/")
###################### need to change end
# os.listdir(ct_scan_dir)
ct_name, ang_deg, theta, ind_list = get_ind_list(os.listdir(ct_scan_dir))
proj180_ind = find_proj180_ind(ang_deg)[0]
print('Found index of 180 degree projections: ', proj180_ind)
print('Loading CT projections...')
proj = read_tiff_stack(fdir=ct_scan_dir, fname=ct_name)
print(proj.shape)
# plt.imshow(proj[0])
# plt.colorbar()
# plt.show()

# Open Beam (OB)

In [ ]:
# ob_loc = "/HFIR/CG1D/IPTS-"+ipts+"/raw/ob"
ob_loc = "/Users/j35/IPTS/HFIR/CG1D/IPTS-27158/raw/ob"
os.listdir(ob_loc)

In [ ]:
###################### need to change start
#ob_dir = os.path.join(ob_loc, "Aug24_2020")
ob_dir = os.path.join(ob_loc, "2021_10_05_cells")
###################### need to change end
# os.listdir(ob_dir)
ob_name, ob_ind_list = get_list(os.listdir(ob_dir))
print("Loading Open Beam (OB)...")
ob = read_tiff_stack(fdir=ob_dir, fname=ob_name)
print(ob.shape)
# plt.imshow(ob[0])
# plt.colorbar()
# plt.show()

# Dark Current (DC)

In [ ]:
# dc_loc = "/HFIR/CG1D/IPTS-"+ipts+"/raw/df"
dc_loc = "/Users/j35/IPTS/HFIR/CG1D/IPTS-27158/raw/df"
os.listdir(dc_loc)

In [ ]:
###################### need to change start
# dc_dir = os.path.join(dc_loc, "Aug24_2020")
dc_dir = os.path.join(dc_loc, "2021_10_05_cells")
###################### need to change end
# os.listdir(dc_dir)
dc_name, dc_ind_list = get_list(os.listdir(dc_dir))
print("Loading Dark Current (DC)...")
dc = read_tiff_stack(fdir=dc_dir, fname=dc_name)
print(dc.shape)
# plt.imshow(dc[0])
# plt.colorbar()
# plt.show()

# Detect and crop the slits

In [ ]:
slit_box_corners = tomopy.prep.alignment.find_slits_corners_aps_1id(img=ob[0], method='simple')
print("Slit corners: ", slit_box_corners)
proj = tomopy.prep.alignment.remove_slits_aps_1id(proj, slit_box_corners)
ob = tomopy.prep.alignment.remove_slits_aps_1id(ob, slit_box_corners)
dc = tomopy.prep.alignment.remove_slits_aps_1id(dc, slit_box_corners)
print("Projection at 0 degree:")
plt.imshow(proj[0])
plt.show()
print("Projection at 180 degree:")
plt.imshow(proj[proj180_ind])
plt.show()

# Further Define the ROI

In [ ]:
roi_corners = set_roi(corners=slit_box_corners, xmin=250, ymin=600, xmax=1250, ymax=1300)
proj_crop = tomopy.prep.alignment.remove_slits_aps_1id(proj, roi_corners)
ob_crop = tomopy.prep.alignment.remove_slits_aps_1id(ob, roi_corners)
dc_crop = tomopy.prep.alignment.remove_slits_aps_1id(dc, roi_corners)
print("Projection at 0 degree:")
plt.imshow(proj_crop[0])
plt.show()
print("Projection at 180 degree:")
plt.imshow(proj_crop[proj180_ind])
plt.show()

# Remove Outliers

In [ ]:
proj_crop = tomopy.misc.corr.remove_outlier(proj_crop, 50)
plt.imshow(proj_crop[0])
plt.colorbar()
plt.show()

# Normalization

In [ ]:
proj_norm = tomopy.normalize(proj_crop, ob_crop, dc_crop)
plt.imshow(proj_norm[0], vmin=0, vmax=1)
plt.colorbar()
plt.show()

# Beam fluctuation correction

In [ ]:
proj_norm = tomopy.prep.normalize.normalize_bg(proj_norm, air=50)
plt.imshow(proj_norm[0], vmin=0, vmax=1)
plt.colorbar()
plt.show()

# Minus log conversion

In [ ]:
proj_mlog = tomopy.minus_log(proj_norm)
plt.imshow(proj_mlog[0])
plt.colorbar()
plt.show()

# Ring Artifact Removal

In [ ]:
proj_rmv = tomopy.prep.stripe.remove_all_stripe(proj_mlog)
print("Before ring removal:")
plt.imshow(proj_mlog[:, 500, :])
plt.colorbar()
plt.show()
print("After ring removal:")
plt.imshow(proj_rmv[:, 500, :])
plt.colorbar()
plt.show()

In [ ]:
# # bm3d, better, but slow
# proj_bm3d_norm = bm3d_rmv.extreme_streak_attenuation(proj_mlog)
# proj_bm3d = bm3d_rmv.multiscale_streak_removal(proj_bm3d_norm)

# Find and correct tilt

In [ ]:
print("Finding the tilt angle...")
tilt_ang = tilt.calculate_tilt(image0=proj_rmv[0], image180=proj_rmv[proj180_ind])
print("Found the tilt angle: ", tilt_ang.x)
proj_tilt = tilt.apply_tilt_correction(proj_rmv, tilt_ang.x)
print("Projection at 0 degree:")
plt.imshow(proj_tilt[0])
plt.colorbar()
plt.show()
print("Projection at 180 degree:")
plt.imshow(proj_tilt[proj180_ind])
plt.colorbar()
plt.show()

# Find Center of Rotation

In [ ]:
rot_center = tomopy.find_center_pc(np.squeeze(proj_tilt[0,:,:]), np.squeeze(proj_tilt[proj180_ind,:,:]), tol=0.5)
print('Estimated center of rotation', rot_center)

# Reconstruction

In [ ]:
recon = tomopy.recon(proj_tilt, theta, center=rot_center, algorithm='gridrec', sinogram_order=False)
recon = tomopy.circ_mask(recon, axis=0, ratio=0.95)
plt.imshow(recon[500, :, :])
plt.colorbar()
plt.show()

# Export recon files

In [ ]:
################ change save path for your own
save_to = "/HFIR/CG1D/IPTS-"+ipts+"/shared/autoreduce/rockit_lava"
fname = save_to + "/recon" 
dxchange.write_tiff_stack(recon, fname=fname, overwrite=True)